In [19]:
%pip install opencv-python

     ---------------------------------------- 35.6/35.6 MB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
## create data augmentation function that exsposes 10 random examples of the input image


# Exploring data in csv file

In [44]:
import pandas as pd
import numpy as np

census_train = pd.read_csv('./data/census/train.csv', index_col=False) 
census_test = pd.read_csv('./data/census/test.csv', skiprows=1, index_col=False) 


census_train = census_train.dropna(how="any", axis=0)
census_test = census_test.dropna(how="any", axis=0)

#you turn! Analyise data
# get hash unique values
# get age unique values

In [45]:
import pandas as pd

from tensorflow.python.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from tensorflow.python.feature_column import feature_column_v2 as fc

from tensorflow import feature_column
#import tensorflow.compat.v1 as tf1
import math


In [46]:
census_train['income'] = census_train['income'].map(lambda x: x == '>50K')
census_train['income'] = census_train['income'].astype(float)
census_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0.0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0.0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0.0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0.0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0.0


In [47]:
train, test = train_test_split(census_train, test_size=0.1)
train, val = train_test_split(train, test_size=0.1)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

26373 train examples
2931 validation examples
3257 test examples


In [77]:
# Convert a Pandas dataframe 
import tensorflow as tf


def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('income')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.repeat().batch(batch_size)
  return ds

In [78]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [79]:
for feature_batch, label_batch in train_ds.take(1):
  print('All features:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['age'])
  print('A batch of labels:', label_batch )

All features: ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
A batch of ages: tf.Tensor([19 28 66 19 60], shape=(5,), dtype=int64)
A batch of labels: tf.Tensor([0. 0. 0. 0. 0.], shape=(5,), dtype=float64)


In [80]:
# A batch of data
# We'll use this to show the output of various
# feature columns.
example_batch = list(train_ds.take(1))[0][0]

In [81]:
# Call a feature column on a batch of data and show the result
def demo(feature_column):
  feature_layer = tf.keras.layers.DenseFeatures([feature_column])
  print(feature_layer(example_batch).numpy())

In [82]:
import tensorflow as tf
age = tf.feature_column.numeric_column("age")
demo(age)

[[22.]
 [60.]
 [23.]
 [57.]
 [23.]]


In [83]:
# A list of the feature columns we'll use to train the Linear model
feature_columns = []

In [84]:
#1- Numerical attributes
##1.1- Bucketizing
import tensorflow as tf

tf.feature_column.bucketized_column

age = tf.feature_column.numeric_column('age')
age_buckets_1 = tf.feature_column.bucketized_column(
    tf.feature_column.numeric_column('age'), 
    boundaries=[31, 46, 60, 75, 90] # specify the ranges
)

age_buckets_2 = tf.feature_column.bucketized_column(
    tf.feature_column.numeric_column('age'),
    list(range(10))
)

feature_columns.append(age_buckets_2)
demo(age_buckets_1)

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [85]:
indices = [0, 1, 2]
depth = 3
tf.one_hot(indices, depth)  # output: [3 x 3]
# [[1., 0., 0.],
#  [0., 1., 0.],
#  [0., 0., 1.]]


<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)>

In [86]:
#1.2
education = tf.feature_column.categorical_column_with_vocabulary_list(
    "education", [
        "Bachelors", "HS-grad", "11th", "Masters", "9th",
        "Some-college", "Assoc-acdm", "Assoc-voc", "7th-8th",
        "Doctorate", "Prof-school", "5th-6th", "10th", "1st-4th",
        "Preschool", "12th"
    ])

feature_columns.append(education)
education_one_hot = fc.indicator_column(education)
demo(education_one_hot)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [87]:
#1.3 hashed bucket
native_country = tf.feature_column.categorical_column_with_hash_bucket('native-country', 1000)
feature_columns.append(native_country)

In [88]:
#1.4 cross column; combine two features
age_cross_education = tf.feature_column.crossed_column(
    [age_buckets_1, education],
    hash_bucket_size=int(1e4) # Using a hash is handy here
)
feature_columns.append(age_cross_education)
crossed_feature = fc.crossed_column([age_buckets_1, education], hash_bucket_size=1000)
demo(fc.indicator_column(crossed_feature))
demo(fc.indicator_column(age_cross_education))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
